In [209]:
import time

def process_datetime(date_time):
    return date_time.strftime(format='%Y-%m-%d')

In [210]:
df_air = pd.read_csv('air_quality.csv')
df_air.columns = ['datetime', 'Year', 'Concentration', 'UNITS','SITE_LATITUDE', 'SITE_LONGITUDE']
df_air['datetime'] = pd.to_datetime(df_air['datetime'], format='%m/%d/%Y')
df_air['datetime'].apply(process_datetime).unique()
df_air.head()

,datetime,Year,Concentration,UNITS,SITE_LATITUDE,SITE_LONGITUDE
0,2008-07-01,2008,16.6,ug/m3 LC,40.816,-73.902
1,2008-07-02,2008,9.6,ug/m3 LC,40.816,-73.902
2,2008-07-03,2008,22.9,ug/m3 LC,40.816,-73.902
3,2008-07-04,2008,25.4,ug/m3 LC,40.816,-73.902
4,2008-07-05,2008,20.1,ug/m3 LC,40.816,-73.902


In [154]:
import requests
def load_historical_data(location, dates):
    '''
    dates: list of date --- yyyy-MM-dd
    return: df_day
            df_hourly
    '''
    interval = 1
    url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx'
    key = 'ddc7eb4778ba48e69e825031190612'
    fm = 'json'

    days_data = []
    days_hourly_data = []

    for day in dates:
        paras = dict()
        paras['key'] = key
        paras['format'] = fm
        paras['q'] = location
        paras['date'] = day
        paras['tp'] = interval
        
        r = requests.get(url, paras)
        r.raise_for_status()                             ### try and except? ### 

        city = r.json()['data']['request'][0]['query']
        result = r.json()['data']['weather'][0]
        
        #################### day data ####################
        day_data = []
        day_data.append(city)

        day_data.append(pd.to_datetime(day, format='%Y-%m-%d'))
        day_data.append(result['astronomy'][0]['sunrise'])
        day_data.append(result['astronomy'][0]['sunset'])
        day_data.append(result['astronomy'][0]['moonrise'])
        day_data.append(result['astronomy'][0]['moonset'])
        day_data.append(result['astronomy'][0]['moon_phase'])
        day_data.append(result['astronomy'][0]['moon_illumination'])
        day_data.append(result['maxtempC'])
        day_data.append(result['maxtempF'])
        day_data.append(result['mintempC'])
        day_data.append(result['mintempF'])
        day_data.append(result['avgtempC'])
        day_data.append(result['avgtempF'])
        day_data.append(result['totalSnow_cm'])
        day_data.append(result['sunHour'])
        day_data.append(result['uvIndex'])
        
        days_data.append(day_data)
        

    df_day = pd.DataFrame(days_data, columns = ['city', 'datetime', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination',
                                                'maxtempC', 'maxtempF', 'mintempC', 'mintempF', 'avgtempC', 'avgtempF', 'totalSnow_cm',
                                                'sunHour', 'uvIndex'])
    df_day.to_csv('daily_data.csv')
    return df_day

In [216]:
date_list = df['datetime'].apply(process_datetime).unique()
date_list = date_list.tolist()
location = 'New+York'
data = load_historical_data(location, date_list)
data.to_csv('daily_data.csv')

In [218]:
df_daily = pd.read_csv('daily_data.csv')
df_daily['datetime'] = pd.to_datetime(df_daily['datetime'], format='%Y/%m/%d')
df_daily['datetime'].apply(process_datetime)
df_daily.head()

,datetime,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex
0,2008-07-01,4:29 AM,7:31 PM,2:24 AM,6:37 PM,Waning Crescent,4,0,32,0,32,19,66,0.0,14.5,1
1,2008-07-02,4:29 AM,7:31 PM,3:30 AM,7:37 PM,New Moon,0,0,32,0,32,18,65,0.0,14.5,1
2,2008-07-03,4:30 AM,7:31 PM,4:45 AM,8:24 PM,New Moon,0,0,32,0,32,19,66,0.0,14.5,1
3,2008-07-04,4:30 AM,7:31 PM,6:04 AM,9:01 PM,Waxing Crescent,3,0,32,0,32,21,70,0.0,12.5,1
4,2008-07-05,4:31 AM,7:30 PM,7:22 AM,9:31 PM,Waxing Crescent,10,0,32,0,32,20,67,0.0,14.5,1


In [219]:
dfm = df_air.merge(df_daily,how='inner',on='datetime')
dfm.head()

,datetime,Year,Concentration,UNITS,SITE_LATITUDE,SITE_LONGITUDE,sunrise,sunset,moonrise,moonset,...,moon_illumination,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex
0,2008-07-01,2008,16.6,ug/m3 LC,40.816,-73.902,4:29 AM,7:31 PM,2:24 AM,6:37 PM,...,4,0,32,0,32,19,66,0.0,14.5,1
1,2008-07-02,2008,9.6,ug/m3 LC,40.816,-73.902,4:29 AM,7:31 PM,3:30 AM,7:37 PM,...,0,0,32,0,32,18,65,0.0,14.5,1
2,2008-07-03,2008,22.9,ug/m3 LC,40.816,-73.902,4:30 AM,7:31 PM,4:45 AM,8:24 PM,...,0,0,32,0,32,19,66,0.0,14.5,1
3,2008-07-04,2008,25.4,ug/m3 LC,40.816,-73.902,4:30 AM,7:31 PM,6:04 AM,9:01 PM,...,3,0,32,0,32,21,70,0.0,12.5,1
4,2008-07-05,2008,20.1,ug/m3 LC,40.816,-73.902,4:31 AM,7:30 PM,7:22 AM,9:31 PM,...,10,0,32,0,32,20,67,0.0,14.5,1


In [220]:
def enhancement():
    return html.Div([
        html.Div([

            html.Div([
                dcc.Dropdown(
                    id='xaxis-column',
                    options=[{'label': i, 'value': i} for i in ['maxtempC', 'maxtempF', 'mintempC',
                                                                'mintempF', 'avgtempC', 'avgtempF',
                                                                'totalSnow_cm', 'sunHour','uvIndex']],
                    value=''
                ),
            ],
            style={'width': '48%', 'display': 'inline-block'}),
        ]),

        dcc.Graph(id='indicator-graphic'),

        dcc.Slider(
            id='year--slider',
            min=df['Year'].min(),
            max=df['Year'].max(),
            value=df['Year'].max(),
            marks={str(year): str(year) for year in dfm['Year'].unique()},
            step=None
        )
    ])


In [ ]:
@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('year--slider', 'value')])
def update_graph(xaxis_column_name, year_value):
    dff = dfm[dfm['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[xaxis_column_name],
            y=dff['Concentration'],
            text=dff['datetime'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
            },
            yaxis={
                'title': 'PM2.5 Concentration',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }